In [64]:
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import env
import os
pd.set_option('display.max_columns', None)

In [17]:
# function to establish connection to MySQL workbench to retrieve data.
def get_connection(db, user=env.username, host=env.host, password=env.password):
    return f'mysql+pymysql://{env.username}:{env.password}@{env.host}/{db}'

In [46]:
def new_zillow_data():
    sql_querry= '''
            select *
            from properties_2017
            right join predictions_2017  using(parcelid)
            WHERE transactiondate < '2018'
                  
            '''
    
    
    df= pd.read_sql(sql_querry, get_connection('zillow'))
    df= df.drop(columns='id')
    df= df.drop_duplicates(subset=['parcelid'],keep='last')
    return df

    

In [47]:
def zillow_data():
    '''this function returns the zillow data and creates the csv file in local directory  if it doesnot exist already.'''
    filename= "zillow.csv"
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read data from database in dataframe
        df= new_zillow_data()
        #cache data
        df.to_csv(filename)
        return df

In [48]:
df= zillow_data()
df.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,261.0,None,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,None,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13
2,14186244,0.005383,2017-01-01,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,261.0,None,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13
3,12177905,-0.103410,2017-01-01,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13
4,10887214,0.006940,2017-01-01,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,1.0,010C,266.0,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,None,NaN,6.037124e+13


In [49]:
#shape of the dataframe
df.shape

(77414, 60)

# Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)



In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77414 entries, 0 to 77612
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77414 non-null  int64  
 1   logerror                      77414 non-null  float64
 2   transactiondate               77414 non-null  object 
 3   airconditioningtypeid         24953 non-null  float64
 4   architecturalstyletypeid      206 non-null    float64
 5   basementsqft                  50 non-null     float64
 6   bathroomcnt                   77381 non-null  float64
 7   bedroomcnt                    77381 non-null  float64
 8   buildingclasstypeid           15 non-null     float64
 9   buildingqualitytypeid         49672 non-null  float64
 10  calculatedbathnbr             76772 non-null  float64
 11  decktypeid                    614 non-null    float64
 12  finishedfloor1squarefeet      6023 non-null   float64
 13  c

In [8]:
df.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,5.232000e+04,13615.000000,70.00000,47.000000,52320.000000,52320.000000,33655.000000,52185.000000,389.0,4371.000000,52239.000000,52074.000000,4371.000000,165.000000,52320.00000,7233.000000,52185.000000,17984.000000,17984.000000,1514.0,33850.000000,5.232000e+04,5.232000e+04,5.195400e+04,11077.0,865.000000,443.0,1071.0,9990.0,52320.0,5.232000e+04,51284.000000,52320.000000,18981.000000,52294.000000,52320.000000,47.0,6717.000000,76.000000,33762.000000,1935.000000,63.000000,52206.000000,14532.000000,81.0,5.223800e+04,5.231900e+04,52320.0,5.231900e+04,52316.000000,2068.000000,5.219900e+04,52320.000000
mean,1.299682e+07,2.439589,7.10000,678.978723,2.300010,3.300765,6.264894,2.305270,66.0,1546.288035,1923.152702,1925.313899,1567.099291,1241.078788,6049.13555,1.209733,2.240031,1.949066,495.927046,1.0,3.965613,3.402256e+07,-1.181941e+08,1.132335e+04,1.0,518.305202,1.0,1.0,1.0,261.0,6.049447e+07,33483.826691,2524.320623,180240.600232,96636.243603,1.894457,7.0,1.013697,5.973684,1.000859,337.359690,225.619048,1963.421254,1.414189,1.0,1.966362e+05,5.298235e+05,2016.0,3.334917e+05,6454.743137,14.101064,6.050245e+13,0.018131
std,3.350919e+06,3.847925,2.66567,711.825226,1.022807,0.947555,1.716346,1.018067,0.0,675.339282,1004.045419,1003.952951,745.084810,776.959074,21.02913,0.512946,0.993303,0.549974,157.100947,0.0,2.562660,2.740624e+05,3.567149e+05,8.674164e+04,0.0,156.614794,0.0,0.0,0.0,0.0,2.086892e+05,49116.149883,804.098580,159928.810868,4610.382493,3.146952,0.0,0.142690,0.229416,0.030290,249.579315,188.505106,23.125670,0.540685,0.0,2.542862e+05,7.518296e+05,0.0,5.705106e+05,8752.477267,2.400399,1.861127e+12,0.176905
min,1.071186e+07,1.000000,2.00000,38.000000,0.000000,0.000000,1.000000,1.000000,66.0,184.000000,128.000000,128.000000,184.000000,380.000000,6037.00000,1.000000,1.000000,0.000000,0.000000,1.0,1.000000,3.334062e+07,-1.194754e+08,2.360000e+02,1.0,24.000000,1.0,1.0,1.0,261.0,6.037101e+07,3491.000000,1286.000000,6952.000000,95982.000000,0.000000,7.0,1.000000,4.000000,1.000000,11.000000,12.000000,1878.000000,1.000000,1.0,1.290000e+02,1.000000e+03,2016.0,1.610000e+02,49.180000,4.000000,6.037101e+13,-4.655420
25%,1.151018e+07,1.000000,7.00000,263.500000,2.000000,3.000000,5.000000,2.000000,66.0,1151.000000,1268.000000,1270.000000,1152.000000,784.000000,6037.00000,1.000000,2.000000,2.000000,426.000000,1.0,2.000000,3.382690e+07,-1.184009e+08,5.583000e+03,1.0,424.000000,1.0,1.0,1.0,261.0,6.037401e+07,12447.000000,1286.000000,41131.000000,96206.000000,0.000000,7.0,1.000000,6.000000,1.000000,200.000000,71.500000,1950.000000,1.000000,1.0,7.715900e+04,1.940330e+05,2016.0,7.619400e+04,2660.982500,14.000000,6.037401e+13,-0.024707
50%,1.257829e+07,1.000000,7.00000,512.000000,2.000000,3.000000,6.000000,2.000000,66.0,1410.000000,1659.000000,1661.000000,1415.000000,1008.000000,6037.00000,1.000000,2.000000,2.000000,462.000000,1.0,2.000000,3.402375e+07,-1.181531e+08,6.841000e+03,1.0,500.000000,1.0,1.0,1.0,261.0,6.037620e+07,24812.000000,3101.000000,118208.000000,96412.000000,0.000000,7.0,1.000000,6.000000,1.000000,280.000000,180.000000,1961.000000,1.000000,1.0,1.319050e+05,3.740060e+05,2016.0,2.180790e+05,4650.56500

In [9]:
# value counts for each colummn in the dataframe
for col in df.columns:
    print(col)
    print(df[col].value_counts())
         
        

parcelid
14297519    1
12895331    1
12680821    1
11839030    1
17148095    1
           ..
17294679    1
14138000    1
12026029    1
14212835    1
12826780    1
Name: parcelid, Length: 52320, dtype: int64
airconditioningtypeid
1.0     11873
13.0     1567
5.0       159
11.0       16
Name: airconditioningtypeid, dtype: int64
architecturalstyletypeid
7.0     62
3.0      3
2.0      2
21.0     2
8.0      1
Name: architecturalstyletypeid, dtype: int64
basementsqft
900.0     2
640.0     2
100.0     2
515.0     2
273.0     2
912.0     2
314.0     1
819.0     1
1809.0    1
604.0     1
126.0     1
588.0     1
786.0     1
669.0     1
800.0     1
396.0     1
112.0     1
645.0     1
384.0     1
1969.0    1
252.0     1
600.0     1
1218.0    1
280.0     1
300.0     1
224.0     1
512.0     1
380.0     1
204.0     1
200.0     1
90.0      1
3112.0    1
405.0     1
1252.0    1
254.0     1
3560.0    1
352.0     1
168.0     1
394.0     1
1416.0    1
38.0      1
Name: basementsqft, dtype: int64
bathroomcn

Name: structuretaxvaluedollarcnt, Length: 37333, dtype: int64
taxvaluedollarcnt
455000.0    33
600000.0    30
500000.0    24
550000.0    24
450000.0    24
            ..
690763.0     1
379764.0     1
317155.0     1
155695.0     1
49546.0      1
Name: taxvaluedollarcnt, Length: 38942, dtype: int64
assessmentyear
2016.0    52320
Name: assessmentyear, dtype: int64
landtaxvaluedollarcnt
21299.0     95
22755.0     63
30210.0     63
16522.0     53
16749.0     51
            ..
86973.0      1
499170.0     1
250803.0     1
122033.0     1
283704.0     1
Name: landtaxvaluedollarcnt, Length: 36912, dtype: int64
taxamount
345.72     4
2914.16    3
1404.44    3
955.88     3
4012.74    3
          ..
1043.94    1
6526.81    1
2821.20    1
4626.40    1
6317.15    1
Name: taxamount, Length: 50834, dtype: int64
taxdelinquencyflag
Y    2068
Name: taxdelinquencyflag, dtype: int64
taxdelinquencyyear
15.0    1124
14.0     559
13.0     126
12.0      81
11.0      79
10.0      47
9.0       33
8.0        8
7.0

In [12]:
# nulls by columns
df.isnull().head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,False,True,True,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False
1,False,True,True,True,False,False,True,True,False,True,False,False,False,True,True,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False
2,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,False,True,True,True,False,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False
3,False,True,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,True,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False
4,False,False,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False


In [14]:
null_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(),
                        'pct_rows_missing': (df.isnull().sum()/ df.shape[0])})
null_col.head()

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
airconditioningtypeid,38705,0.739774
architecturalstyletypeid,52250,0.998662
basementsqft,52273,0.999102
bathroomcnt,0,0.000000


# Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [51]:
def nulls_by_col(df):
    num_missing= df.isnull().sum()
    rows= df.shape[0]
    pct_missing = num_missing / rows
    cols_missing = pd.DataFrame({'num_rows_missing':num_missing,'pct_rows_missing':pct_missing})
                                
    return cols_missing.sort_values(by='num_rows_missing',ascending=False)

In [52]:
nulls_by_col(df).head()

,num_rows_missing,pct_rows_missing
buildingclasstypeid,77399,0.999806
finishedsquarefeet13,77373,0.999470
basementsqft,77364,0.999354
storytypeid,77364,0.999354
yardbuildingsqft26,77344,0.999096


# 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values. 

In [44]:
def nulls_by_rows(df):
    
    num_missing= (df.isnull().sum(axis=1))
    pct_missing = num_missing / df.shape[1]
    rows_missing = pd.DataFrame({'num_rows_missimg':'num_cols_missing': num_missing, 'percent_cols_missing': pct_missing,'num_rows_missimg':})
    
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)
    

In [53]:
nulls_by_rows(df)

,num_cols_missing,percent_cols_missing
40789,57,0.950000
19758,57,0.950000
388,57,0.950000
64308,57,0.950000
8888,57,0.950000
...,...,...
9659,20,0.333333
62437,20,0.333333
54572,20,0.333333
18965,19,0.316667


# Mall  customers

In [21]:
# ACQUIRE mall data from server
sql= '''Select * from customers'''


In [22]:
df= pd.read_sql(sql,get_connection('mall_customers'))
df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [24]:
#summarize
df.describe()

,customer_id,age,annual_income,spending_score
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [25]:
# check nulls
df.isna().sum()

customer_id       0
gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

### Outliers using IQR

In [28]:
# age quantiles
q1, q3 = df['age'].quantile([.25,.75])
q1, q3

(28.75, 49.0)

In [29]:
# calculating IQR
age_iqr = q3-q1
age_iqr

20.25

In [30]:
# upperbound and lower bound with k =1.5
upper = q3 +(age_iqr * 1.5)
lower = q1 - (age_iqr * 1.5)

upper, lower

(79.375, -1.625)

In [60]:
# confirming age outliers has been handled
df[df.age> upper]

,customer_id,gender,age,annual_income,spending_score


In [61]:
# splitting the data into train, test and validate
seed = 123

train, test_val = train_test_split(df, train_size=0.7,
                                   random_state=seed)

test, val = train_test_split(test_val, train_size=0.5,
                                       random_state=seed)

train.shape, val.shape, test.shape

((140, 5), (30, 5), (30, 5))

In [62]:
# encoding categorical column using pd.get_dummies
train =pd.get_dummies(train,drop_first= True)
train.head()

,customer_id,age,annual_income,spending_score,gender_Male
81,82,38,54,55,1
107,108,54,63,46,1
112,113,38,64,42,0
145,146,28,77,97,1
8,9,64,19,3,1


In [63]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 81 to 109
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   customer_id     140 non-null    int64
 1   age             140 non-null    int64
 2   annual_income   140 non-null    int64
 3   spending_score  140 non-null    int64
 4   gender_Male     140 non-null    uint8
dtypes: int64(4), uint8(1)
memory usage: 5.6 KB


In [66]:
# scale the data. columns to scale age and annual income using minmaxsclaer
scaler= MinMaxScaler()
train[['age','annual_income']]= scaler.fit_transform(train[['age','annual_income']])
train.head()


,customer_id,age,annual_income,spending_score,gender_Male
81,82,0.384615,0.319672,55,1
107,108,0.692308,0.393443,46,1
112,113,0.384615,0.401639,42,0
145,146,0.192308,0.508197,97,1
8,9,0.884615,0.032787,3,1
